## Exercise 1

Implement the following function to calculate gini_impurity.

**HINT**

You can count the number of instance of an item in a list like this: 

In [12]:
fruit = ['apple','pear','banana','apple']
fruit.count('apple')

2

In [ ]:
import pandas as pd
import numpy as np

def gini_impurity(groups, classes):
    """Calculate gini impurity for a list of groups
    
    Parameters:
    groups -- A list of lists, where each inner list contains the class values for that group
              For example: [[True,False,True], [False,False,True]]
    classes -- A list of possible class values, e.g. [True, False] or ['Yes', 'No']
    
    Returns:
    float -- The gini impurity score
    """
   
    gini = 0.0     
    return gini


gini_impurity([[True,False,True], [False,False,True]],[True, False])

#  Should equal 0.4444

## Exercise 2

Consider the following data:

| Tid | Refund | Marital Status | Taxable Income (K) | Cheat |
|-----|--------|----------------|--------------------|-------|
| 1   | Yes    | Single         | 125                | No    |
| 2   | No     | Married        | 100                | No    |
| 3   | No     | Single         | 70                 | No    |
| 4   | Yes    | Married        | 120                | No    |
| 5   | No     | Divorced       | 95                 | Yes   |
| 6   | No     | Married        | 60                 | No    |
| 7   | Yes    | Divorced       | 220                | No    |
| 8   | No     | Single         | 85                 | Yes   |
| 9   | No     | Married        | 75                 | No    |
| 10  | No     | Single         | 90                 | Yes   |

Use the gini impurity method you implemented above to find the best split in the data.  You'll need to complete the "find_best_split" function.

First, we set up the data...

In [2]:
import pandas as pd
import numpy as np

# Sample data provided
data = {
    'Refund': ['Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No'],
    'Marital Status': ['Single', 'Married', 'Single', 'Married', 'Divorced', 'Married', 'Divorced', 'Single', 'Married', 'Single'],
    'Taxable Income (K)': [125, 100, 70, 120, 95, 60, 220, 85, 75, 90],
    'Cheat': ['No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes']
}

# Create a DataFrame
df = pd.DataFrame(data)


Implement the following function to calculate gini impurity...

In [3]:

# Function to calculate Gini impurity
# Here, groups is going to just be a list of 
def gini_impurity(groups, classes):
    # IMPLEMENT ME!!!
    gini = None
    return gini


The following helper function splits a data into two sets of rows based on a column value.

In [11]:
def split_data(df, column, split_value, target):
    """Split a dataframe into two groups based on a column and value
    
    Parameters:
    df -- The pandas dataframe
    column -- The column name to split on
    split_value -- The value to split on
    target - the column with the values to extract
    
    Returns:
    tuple -- Two lists containing the target values for each group
    """
    if pd.api.types.is_numeric_dtype(df[column]):
        mask = df[column] <= split_value
    else:
        mask = df[column] == split_value
        
    left_group = df[mask][target].tolist()
    right_group = df[~mask][target].tolist()
    
    return left_group, right_group



The following cell can be used to generate candidate splits for continuous columns.

In [ ]:


# Function to get quartiles for continuous variables
def get_quartiles(series):
    return np.percentile(series, [25, 50, 75])


Ok, you need to complete the following!

In [ ]:

# Function to find the best split given a target column
def find_best_split(df,target):
    """Find the best split in a dataframe
    
    Parameters:
    df -- The pandas dataframe
    target -- The column to use as the output target
    
    Returns:
    dict -- The best split information
    """
    class_values = list(df[target].unique())
    best_column = None
    best_value = None
    best_gini = float('inf')
    best_groups = None
    
    # Try each column except the target
    for column in [c for c in df.columns if c != target]:
        if pd.api.types.is_numeric_dtype(df[column]):
            # For numeric columns, try quartile splits
            split_candidates = get_quartiles(df[column])
        else:
            # For categorical columns, try each unique value
            split_candidates = df[column].unique()
            
        for value in split_candidates:
            pass
            # FILL IN THE CODE HERE
            # This code should:
            # -- Go through each split candidate
            # -- Calculate gini_impurity
            # -- If its better than the best known thus far, 
            # -- Update best_column, best_value, best_gini, and best_groups
    
    return {
        'column': best_column,
        'value': best_value,
        'gini': best_gini,
        'groups': best_groups
    }


Finally, find the best split.

In [ ]:
best_split = find_best_split(df,target)
print('Best Split: Column', best_split['column'], 'with value', best_split['value'], 'gini score =', best_split['gini'])

## Exercise 3

Use the following to calculate the entire decision tree!  Evaluate the decision tree by hand using accuracy.

In [ ]:
def build_tree(df, target, depth=0, max_depth=3, min_samples=2):
    """Build and print a decision tree using recursive splitting
    
    Parameters:
    df -- The pandas dataframe
    depth -- Current depth in the tree (used for indentation)
    max_depth -- Maximum depth to grow the tree
    min_samples -- Minimum samples required to split
    """
    # If all samples have same class, print the result and stop
    if len(df['Cheat'].unique()) == 1:
        print('    ' * depth + f"Predict: {df[target].iloc[0]}")
        return
        
    # If we hit max depth or min samples, print majority class and stop
    if depth >= max_depth or len(df) < min_samples:
        majority_class = df[target].mode().iloc[0]
        print('    ' * depth + f"Predict: {majority_class}")
        return
        
    # Find the best split
    split = find_best_split(df,target)
    column = split['column']
    value = split['value']
    
    # If we couldn't find a good split, stop here
    if column is None:
        majority_class = df[target].mode().iloc[0]
        print('    ' * depth + f"Predict: {majority_class}")
        return
    
    # Print this split decision
    if pd.api.types.is_numeric_dtype(df[column]):
        print('    ' * depth + f"If {column} <= {value}:")
    else:
        print('    ' * depth + f"If {column} == {value}:")
    
    # Split the data
    if pd.api.types.is_numeric_dtype(df[column]):
        left_mask = df[column] <= value
    else:
        left_mask = df[column] == value
        
    # Recursively build tree for each split
    left_data = df[left_mask]
    right_data = df[~left_mask]
    
    # Process left branch
    build_tree(left_data, target, depth + 1, max_depth, min_samples)
    
    # Print the "else" for this split
    if pd.api.types.is_numeric_dtype(df[column]):
        print('    ' * depth + f"If {column} > {value}:")
    else:
        print('    ' * depth + f"If {column} != {value}:")
    
    # Process right branch
    build_tree(right_data, target, depth + 1, max_depth, min_samples)
